In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import re
import os
from itertools import izip
import itertools
from collections import defaultdict
from gensim.parsing import preprocessing as gprocessing
from spacy.en import English as nlp
from bs4 import BeautifulSoup

In [2]:
os.chdir("..")
os.getcwd()

'/mnt/Storage/Coding_Projects/Candidate_Classifier'

## Corpora

In [6]:
from candidate_classifier.nltk_model import NgramModel
from candidate_classifier import utils
from nltk.probability import LaplaceProbDist, LidstoneProbDist
from nltk.corpus import PlaintextCorpusReader
import os

In [7]:
from nltk.corpus import switchboard, nps_chat
from nltk.corpus.reader.util import *
from nltk.tokenize import *
import nltk

In [8]:
corpus_root = os.path.abspath('candidate_classifier/data')

In [48]:
debates = DebateCorpusReader('candidate_classifier/data', '.*', word_tokenizer=DummyTokenizer())

### MODELS!!!

In [5]:
from candidate_classifier.nltk_model import NgramModel
from nltk.probability import LaplaceProbDist, LidstoneProbDist, SimpleGoodTuringProbDist
from nltk.corpus import PlaintextCorpusReader

In [169]:
est = lambda freqdist, bins: LidstoneProbDist(freqdist, 0.2, bins)

In [6]:
e2 = lambda freqdist, bins: SimpleGoodTuringProbDist(freqdist, bins)

In [18]:
s1 = "This is a test.".split()
s2 = "I'm starting to know what God felt like when he sat out there in the darkness, creating the world.".split()

text = (s for s in [s1, s2])

In [19]:
model = NgramModel(3, text, estimator=e2, bins=len(set(flatten([s1, s2]))))

AssertionError: bins parameter must not be less than 23=freqdist.B()+1

In [17]:
model.generate(10)

[u'.', u'.', u'.', u'.', u'.', u'.', u'.', u'.', u'.', u'.']

In [219]:
models = {}

for k, v in debates.joined_speaker_paras().iteritems():
    if stats[k]['num_word_types'] >= 100:
#         docs = (concat(chunk) for chunk in itertools.islice((sent.split() for sent in flatten(v)), 10))
        docs = (' '.join(chunk).split() for chunk in chunker(flatten(v)))
#         print list(docs)[:2]
        models[k] = NgramModel(3, docs, estimator=est, bins=stats[k]['num_word_types'])

In [220]:
for i in range(5):
    print ' '.join(models['TRUMP'].generate(50)), '\n'

<S> so, look, we have a tremendous success. </S> <S> well, i'd like to respond, i'd like to tell you, we are spending $200 billion a year imbalance. </S> <S> assad is fighting isis. </S> <S> we cannot lose this election. </S> <S> but i say that i have used 

<S> second of all, let me — caesar's, the rolls-royce, as you too know, i know i built a very good people are going to find a finer men. </S> <S> but a lot — but the democrats are going to even say — i think, for me, nuclear is 

<S> i agree 100 percent, dana. </S> <S> mr. trump" — these are terrible people in actually 15 — going to have people come into our country has right now by people that want to endorse me. </S> <S> you feel safe right now? </S> <S> i spoke to a 

<S> it's hurting us from every standpoint. </S> <S> are you going to embarrass, but virtually every person that you wouldn't have millions of jobs and manufacturing back. </S> <S> one thing at a time. </S> <S> so we can get the senate, but i hope that answers your 

In [221]:
for i in range(5):
    print ' '.join(models['BUSH'].generate(50)), '\n'

<S> if they had — they are barrel bombing the innocents in that country. </S> <S> if we're serious about high growth, then we 'ought to create more government from the federal law. </S> <S> when i'm elected president, and hillary clinton, who can't even project force. </S> <S> to 

<S> yeah. here's — apart from that, there will be — this is recognizing that there will be — this is not a factor. </S> <S> everybody's record's going to win when we — you flew in with your 767, didn't you? </S> <S> imagine what it's like now to 

<S> it's a good job. </S> <S> that expertise needs to complete dialogue with us. </S> <S> so he supports clinton — — the pacific agreement. </S> <S> lay them out there. </S> <S> i just did. </S> <S> i look forward to talking tonight about how bad washington, d.c. 

<S> this would be devastating for agriculture and many of the hhs funding, there is a place where security can take place. </S> <S> 6.5 million people are suffering today in america. </S> <S> no it isn't

In [185]:
[sent.split() for sent in flatten(debates.joined_speaker_paras()['???'])]

[[u'<S>',
  u"that's",
  u"what's",
  u'wrong',
  u'with',
  u'politicians',
  u'in',
  u'washington',
  u'right',
  u'now.',
  u'</S>'],
 [u'<S>',
  u'they',
  u'think',
  u'we',
  u'can',
  u'take',
  u'a',
  u'country',
  u'into',
  u'bankruptcy.',
  u'</S>']]

In [16]:
reduce(lambda x, y: '{}({})'.format(y, x), ['a', 'b', 'c'], 'doc')

'c(b(a(doc)))'

# Scratch

In [77]:
from collections import deque

In [104]:
def rrun(s):
    # base case
    if issubclass(type(s), basestring):
        yield s
    # Recursive case
    else:
        for subelt in s:
            for elt in rrun(subelt):
                yield elt

In [105]:
test = [['1','2'], ['3', '4'], '5', '6']

[s for s in rrun(test)]

['1', '2', '3', '4', '5', '6']

In [13]:
def flatten(nested):
    try:
        try: nested + ''
        except TypeError: pass
        else: raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested
        
print list(flatten(['foo', ['bar', ['baz']]]))

['foo', 'bar', 'baz']


In [109]:
list(flatten([1, 2, 3]))

[1, 2, 3]

In [41]:
[i for i in flatten([((j for j in xrange(i)) for i in xrange(5)), 9, 8])]

[0, 0, 1, 0, 1, 2, 0, 1, 2, 3, 9, 8]

In [112]:
def nested_map(root, func):
    """Takes in an arbitrarily nested iterator and traverses it
     deapth first to create generators at every level, where
     every element has a function applied to it.
     """
    try:
        # Don't unwind strings
        try:
            root + ''
        except TypeError:
            pass
        else:
            raise TypeError
        
        return (nested_map(elt, func) for elt in root)
        
    except TypeError:
        return func(root)

In [116]:
[i for i in nested_map(test, int)]

[<generator object <genexpr> at 0x7f20b18c0230>,
 <generator object <genexpr> at 0x7f20b18c0fa0>,
 5,
 6]

In [142]:
#! /usr/bin/env python2

"""
Idea behind the structure:

 String Level Transformations
   - Encodings
   - Substitutions
 Tokenize
 Sentence Level Transformations
   - Filters
   - Transformations
 Token Level Transformations
   - Filters
   - Substitutions
   - Additions
       - Append POS, lemmatize, etc

"""

import re
import string
# import itertools
from bs4 import BeautifulSoup as bs
from gensim.utils import any2unicode, any2utf8, decode_htmlentities, deaccent
from gensim.parsing import preprocessing as gprocessing
from candidate_classifier import utils


__author__ = 'Eric Lind'


# TODO:
# pre/postfilter substitutions
#  - What if the substitution you perform changes the string so that it would have been filtered?


class StringProcessor(object):
    def __init__(self,
                 string_transformer,
                 sentence_transformer=None,
                 token_transformer=None):
        """Autobots, roll out!"""
        self.str_transformer = string_transformer
        self.sent_transformer = sentence_transformer
        self.tok_transformer = token_transformer

    def __call__(self, s):
        # Process string
        processed = self.str_transformer(s)

        # TODO: Fix this and figure our contract
        # Sentences
        if self.sent_transformer is not None:
            sents = self.sent_transformer(s)

            for sent in sents:
                # Tokens
                if self.tok_transformer is not None:
                    sent = self.tok_transformer(sent)

                yield sent



    # def __call__(self, s):
    #     Yes, I know this is difficult to read.  But this is
    #     my library and I'll do as I please thank you very much.
    #     The contract of this reduce operation is essentially this:
    #     f([a, b, c], 's') ==> c(b(a('s')))
    #     The reason for the reversal is so that when a user specifies
    #     the function they want to call first, it can be first in the list.
        # return reduce(lambda x, y: y(x), self.transformers, s)


# TODO:
# Make a repr that shows the current filters/substitutions being used
# Add the available built-in filters/substitutions to the docstring
class TransformerABC(object):
    """Interface for applying transformations to a string/token"""

    re_type = type(re.compile(r''))
    ws_pattern = re.compile(r"white(?:-|_|\s)?space", re.I)
    non_ascii_pattern = re.compile(r"non(?:-|_|\s)?ascii", re.I)
    h_ents_pattern = re.compile(r"html(?:-|_|\s)?entities", re.I)
    # TODO: Make min_len an optional argument
    min_len = 3

    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 normalize_encoding=True,
                 flatten=True,  # TODO: Document
                 # Tokenization always occurs last.
                 # If you want to process the tokens, use another Transformer
                 # tokenizer must be callable
                 tokenizer=None,
                 **kwargs):
        self.filters = self._process_filters(filters)
        self.pre_substitutions = self._process_substitutions(prefilter_substitutions)
        self.post_substitutions = self._process_substitutions(postfilter_substitutions)
        self.normalize_encoding = normalize_encoding
        self.tokenizer = tokenizer

        # Set processor
        self.processor = self._flat_process if flatten else self._nested_process


    def __call__(self, s):
        # FIXME More documnetation for empty string behavior
        return (i for i in self.processor(s) if i)


    def _process_filters(self, filts):
        """
        Filters are functions that return booleans.  If a function
        resturns True (or some truthy value) for a given string,
        that string is filtered out.
        """
        t = []

        for f in filts:
            # Length
            if f in {'length', 'len'}:
                # Default length is 3
                # Create a closure
                def funcC(fil):
                    def func(s): return len(s) <= self.min_len
                    return func
                t.append(funcC(f))
                # t.append(lambda s: len(s) <= self.min_len)
            elif hasattr(f, '__getitem__'):
                try:
                    if f[0] in {'length', 'len', 'short'}:
                        t.append(lambda s: len(s) <= s[1])
                # FIXME: More thorough checks
                except (IndexError, TypeError):
                    pass

            # Patterns
            elif isinstance(f, self.re_type):
                # Create a closure for the current filter
                def funcC(fil):
                    def func(s): return re.search(fil, s)
                    return func
                t.append(funcC(f))
                # t.append(lambda s: re.search(f, s))

            # Non-Ascii
            elif re.search(self.non_ascii_pattern, str(f)):
                t.append(self._is_non_ascii)

            # Callable
            elif hasattr(f, '__call__'):
                t.append(f)

            # TODO:
            # html tags
            # punctuation

        return t


    def _process_substitutions(self, subs):
        t = []

        # Go through all subs and prepare/apply them.
        # The order is IMPORTANT.  It is expected that
        # the user knows what order they want!
        for sub in subs:
            # HTML
            if sub == 'html':
                t.append(lambda s: bs(s, 'lxml').get_text())

            # HTML entities
            # if sub in {'htmlentities', 'html entities', 'html_entities', 'html-entities'}:
            elif re.search(self.h_ents_pattern, str(sub)):
                t.append(decode_htmlentities)

            # Daccent
            elif sub == 'deaccent':
                t.append(deaccent)

            # Punctuation
            elif sub in {'punct', 'punctuation', 'puncts'}:
                t.append(gprocessing.strip_punctuation)

            # Case
            elif sub == 'lower':
                t.append(string.lower)
            elif sub == 'upper':
                t.append(string.upper)

            # Whitespace
            elif re.search(self.ws_pattern, str(sub)):
                t.append(gprocessing.strip_multiple_whitespaces)
            elif sub == 'strip':
                t.append(string.strip)

            # callable
            elif hasattr(sub, '__call__'):
                t.append(sub)

            # patterns
            elif isinstance(sub, self.re_type):
                # TODO: Make more elegant
                def funcC(sub):
                    def func(s): return sub.sub('', s)
                    return func
                t.append(funcC(sub))
                # t.append(lambda s: sub.sub('', s))
            else:
                try:
                    if isinstance(sub[0], self.re_type):
                        def funcC(sub):
                            # FIXME: Use compiled pattern
                            # def func(s): sub[0].sub(sub[1], s)
                            def func(s): return re.sub(sub[0], sub[1], s)
                            return func
                        t.append(funcC(sub))
                        # t.append(lambda s: re.sub(sub[0], sub[1], s))
                except IndexError:
                    pass

        return t

    @staticmethod
    def _is_non_ascii(s):
        """Returns True if the string contains non-ascii characters
        Fastest way to check for non-ascii:
        http://stackoverflow.com/questions/196345/how-to-check-if-a-string-in-python-is-in-ascii
        """
        try:
            s.decode('ascii')
        except UnicodeDecodeError:
            return True
        else:
            return False

    def _flat_process(self, s):
        """Takes in either an iterator or a string and yields strings.
        Calling this on a nested list will result in a flattened list.
        """
        for elt in utils.flatten(s):
            # print elt
            # Tokenization may add another level of nesting
            for t in utils.flatten(self._process(elt)):
                # print t
                yield t

    def _nested_process(self, s):
        return utils.nested_map(s, self._process)

    def _process(self, s):
        # TODO: Clarify contract return value for filtered strings
        # TODO: Add docs for: if a filter evaluates as False the string is discarded

        # Skip empty strings:
        if len(s) == 0:
            return s

        # Normalize the encoding before anything else
        if self.normalize_encoding:
            s = any2unicode(s)

        # Apply sub, filter, sub
        for sub in self.pre_substitutions:
            s = sub(s)

        # If any of the filters return True, filter the string
        if any(f(s) for f in self.filters):
            return ''

        for sub in self.post_substitutions:
            s = sub(s)

        # Tokenize last.  If you want to process the tokens use another
        # Transformer
        if self.tokenizer:
            s = self.tokenizer(s)

        return s






class StringTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=()):
        super(StringTransformer, self).__init__(prefilter_substitutions,
                                                postfilter_substitutions,
                                                filters)

    def __call__(self, s):
        # Normalize encoding
        # TODO: Find a more robust function for this (e.g. from another library)
        s = any2unicode(s)

        return self._process(s)



class SentenceTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 sent_tokenizer=None):
        super(SentenceTransformer, self).__init__(prefilter_substitutions,
                                                  postfilter_substitutions,
                                                  filters)
        # If no tokenizer is specified, it is assumed that
        # this will be called on a pre-tokenized list of sentences
        self.tokenizer = sent_tokenizer

    def __call__(self, s):
        """Takes in a string and yields sentences"""
        if self.tokenizer is not None:
            sents = self.tokenizer(s)
        else:
            sents = s

        for sent in sents:
            yield self._process(sent)



# TODO:
# - Stopwords
# - Punctuation
# is non-ascii
# is numeric
# is url
# is email address
class TokenTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 tokenizer=None):
        super(TokenTransformer, self).__init__(prefilter_substitutions,
                                               postfilter_substitutions,
                                               filters)
        # self.filters.extend(self._process_token_filters(filters))
        # self.pre_substitutions.extend(self._process_token_substitutions(prefilter_substitutions))
        # self.post_substitutions.extend(self._process_token_substitutions(postfilter_substitutions))

        # If no tokenizer is specified, it is assumed that
        # this will be called on a pre-tokenized list of words
        self.tokenizer = tokenizer

    def _process_token_filters(self, filters):
        # Stopwords
        # if f in {'stop', 'stops', 'stopwords'}:
        #     pass
        pass


    def _process_token_substitutions(self, substitutions):
        pass

    # def __call__(self, sent):
    #     """Takes in a

